+ DB 접속 / 자료 추출(query)
+ 데이터 가공
+ 메일내용 작성
+ 발송

In [1]:
###############
# import
#####

import pandas as pd
from datetime import datetime, timedelta
import requests
import time 
import re
from google.cloud import bigquery
import numpy as np

import os
from os import listdir

import pymysql

# import send_email_v2 as se
# from jinja2 import Template

In [4]:
###############
# DB 접속 / 자료 추출(query)
#####

# 접속 정보 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/boxwood-**************************.json"
client = bigquery.Client()


# 자료1 : 키워드 수집현황
sql1 = """
      SELECT  CHANNEL
      , FORMAT_DATETIME('%m/%d', CAST(A.REG_DTM AS DATETIME)) as DATE
      , FORMAT_DATETIME('%H', CAST(A.REG_DTM AS DATETIME)) as TIME
      , count(*) as cnt
      FROM `boxwood-chassis-356100.online_price.keyword_search_result` A
      where CAST(A.REG_DTM AS DATETIME) between DATE_SUB(current_date(), INTERVAL 2 DAY) and DATE_SUB(current_date(), INTERVAL -1 DAY)
      group by CHANNEL, DATE, TIME
      order by DATE
      """
df1 = client.query(sql1).to_dataframe()



In [6]:
# 자료1 : 키워드 수집현황
sql1 = """
      SELECT *
      FROM `boxwood-chassis-356100.online_price.keyword_search_result` A
      where CAST(A.REG_DTM AS DATETIME) between DATE_SUB(current_date(), INTERVAL 2 DAY) and DATE_SUB(current_date(), INTERVAL -1 DAY)
      and CHANNEL= 'naver_shopping'
      """
df2 = client.query(sql1).to_dataframe()


In [7]:
df2

,KEYWORD,DELIVERY_TYPE,ORDER_INDEX,AD_GUBUN,PRODUCT,PRICE,CHANNEL,REG_DTM,RATING_CNT,PRODUCT_ID,VENDER_ITEM_ID,ITEM_CODE
0,매일유업,3000,1,매일유업 직영스토어,매일두유 99.9 190mlx24,11610,naver_shopping,2022-12-27 14:02:04+00:00,138000,33486735618,,
1,매일유업,3000,2,매일유업 직영스토어,어메이징오트 오리지널 190mlx24 [친환경 각티슈],14310,naver_shopping,2022-12-27 14:02:04+00:00,38640,28943584598,,
2,매일유업,3000,3,매일유업 직영스토어,"소화가잘되는우유(락토프리) 오리지널 멸균 190ml 24팩 [리뷰1,000원]",17500,naver_shopping,2022-12-27 14:02:04+00:00,54849,12548927486,,
3,매일유업,3000,4,매일유업 직영스토어,매일야채 고농축 토마토의힘 125ml 24입,22000,naver_shopping,2022-12-27 14:02:04+00:00,9238,32317867618,,
4,매일유업,3000,1,매일유업 직영스토어,"매일두유 99.9 190mlx24[리뷰1,000원]",11610,naver_shopping,2022-12-26 14:02:04+00:00,137919,33486735618,,
5,매일유업,3000,2,매일유업 직영스토어,"어메이징오트 오리지널 190mlx24 [친환경 각티슈] [리뷰 1,000원]",14310,naver_shopping,2022-12-26 14:02:04+00:00,38552,28943584598,,
6,매일유업,3000,3,매일유업 직영스토어,"소화가잘되는우유(락토프리) 오리지널 멸균 190ml 24팩 [리뷰1,000원]",17500,naver_shopping,2022-12-26 14:02:04+00:00,54756,12548927486,,
7,매일유업,3000,4,매일유업 직영스토어,매일야채 고농축 토마토의힘 125ml 24입,22000,naver_shopping,2022-12-26 14:02:04+00:00,9223,32317867618,,
8,매일유업,3000,25,매일유업 직영스토어,어메이징 오트 바리스타 950ml 3팩,9900,naver_shopping,2022-12-26 14:02:04+00:00,15538,31253551405,,
9,매일유업,0,5,,매일두유 99.9 190ml,10,naver_shopping,2022-12-27 14:02:04+00:00,106418,33486735618,,


In [ ]:
report_01

In [4]:
#########
# 자료2 : 리뷰 수집현황
###
sql2 = """
      SELECT  CHANNEL, FORMAT_DATETIME('%m/%d', CAST(A.DATE AS DATETIME)) as DATE, count(*) as cnt
      FROM `boxwood-chassis-356100.review.review_all` A
      where CAST(A.DATE AS DATETIME) between DATE_SUB(current_date(), INTERVAL 7 DAY) and DATE_SUB(current_date(), INTERVAL -1 DAY)
      group by CHANNEL, FORMAT_DATETIME('%m/%d', CAST(A.DATE AS DATETIME))
      """
df2 = client.query(sql2).to_dataframe()

# 데이터 가공
report_02 = pd.pivot_table(df2, values='cnt', columns=['DATE'], index=[ 'CHANNEL'],  aggfunc='sum').fillna(0).T

In [5]:
report_02

CHANNEL,G마켓,kakaomakers,kurly,카카오 선물하기,카카오 쇼핑하기
DATE,,,,,
09/22,4,65,234,1,18
09/23,4,26,225,1,9
09/24,3,23,181,2,165
09/25,1,25,184,1,60
09/26,5,42,200,1,112
09/27,7,23,204,1,117
09/28,4,44,210,2,76


In [6]:

###############
# 3.이메일 작성
#####

# 이메일 양식(내용) frame 구성하기
message_tmpl="""
<html>
    <!DOCTYPE html>
    <html>
    <head lang ="ko">
      <meta charset ="UTF-8">
      <title>{{title1}}</title>
    </head>
    <style type="text/css">
    table, th, td {
            font-size: 8pt;
            text-align: right;
            border-collapse: collapse;
            font-family: Malgun Gothic;
            white-space: nowrap;
            border: 1px solid #bcbcbc;
            }
    thead tr {
            background-color: #0d47a1;
            color: #ffffff;
        }
    </STYLE>
    <BODY style="FONT-SIZE: 10pt; font-family: 맑은고딕, Malgun Gothic, dotum, gulim, sans-serif">
    안녕하세요,<br>
    <br>
    {{send_date}}시 기준, 채널별 키워드자료 수집 현황 내용 공유드립니다.<br>
    (목적 : 클라우드 수집에 따른 누락여부 확인)<br>
    <br>
    수집된 자료 수 : 최근 1주일<br>
    {{ df_html }}<br>
    <SPAN style="COLOR: #808080">{{add_text1}}</SPAN><br>
    <SPAN style="COLOR: #808080">{{add_text2}}</SPAN><br>
    <br>
    감사합니다.<br>
    <br>
    </BODY>
</html>"""

# 이메일 양식(내용) 변동 내용 정의
render_data = {
    'send_date' : (datetime.today()).strftime('%Y-%m-%d, %H'),
    'df_html' : report_01.to_html(),
    'add_text1' : "*  본 메일은 발신전용입니다.",
    'add_text2' : "** (내용/기술문의 : 고객정보전략담당, 박경하 )"
}


t= Template(message_tmpl)
html_report = t.render(render_data)


In [7]:
# 받는사람 : test 발송
to_users = ['khpark@maeil.com']

# 메일제목
now = datetime.today()    # 현재
date = now.strftime('%Y%m%d')
subject = "[이커머스 TFT] "+date[2:] + "_키워드 수집현황(발송전용)"

# 본문
html = html_report

# 첨부파일 추가
attach = None
graph = None

# 이메일 발송
se.send_email(to_users, subject, html, graph, attach)